In [1]:
%pip install spacy spacy-transformers
%pip install fuzzywuzzy
%pip install python-Levenshtein

  Using cached spacy-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached spacy_transformers-1.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (190 kB)
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
  Using cached thinc-8.1.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (919 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached preshed-3.0.8-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (124 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached murmurhash-1.0.9-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
  Using cached pydantic-2.3.0-py3-none-any.whl (374 kB)
  Using cached srsly-2.4.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl 

In [2]:

!python -m spacy download en_core_web_lg

2023-09-04 07:12:31.456108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 4.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [65]:
import spacy
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
from fuzzywuzzy import fuzz
from collections import defaultdict
import re

In [66]:

def preprocess_string(string : str) -> str:
    string = string.lower().strip()
    string = re.sub(' +', ' ', string)
    string = string.replace("_", " ")
    string = string.replace("-", " ")
    string = string.replace("&", " ")
    string = string.split("(")[0]

    # special cases
    if "computer" in string:
        string = "pc"
    if "tv" in string:
        string = "television"
    if "macbook" in string:
        string = "laptop"
    if "ev" == string:
        string = "car charger"
    
    

    return string

In [79]:
device_list = []
path = "./Energy_graph/data/training_data/"
for file in tqdm(os.listdir(path)):
    if not file.endswith(".pkl"):
        continue
    
    print(file)
    data = pd.read_pickle(path + file)
    for house in data:
        
        for device in data[house]:
            device = preprocess_string(device)
            if device == "aggregate" or "outlet" in device or "sockets" in device or "lamp" in device or "light" in device:
                continue
            test = device
            if "lamp" in device:
                print("HERE: ",house, device , test)
            if device not in device_list:
                device_list.append(device)
    

  8%|▊         | 1/13 [00:00<00:02,  5.95it/s]

REDD.pkl
ECO.pkl


 15%|█▌        | 2/13 [00:03<00:22,  2.05s/it]

REFIT.pkl


 23%|██▎       | 3/13 [00:06<00:23,  2.35s/it]

...

 77%|███████▋  | 10/13 [00:14<00:02,  1.21it/s]

IAWE.pkl
HES.pkl


 85%|████████▍ | 11/13 [00:14<00:01,  1.52it/s]

DRED.pkl


100%|██████████| 13/13 [00:14<00:00,  1.14s/it]


In [82]:

labels = device_list
grouped_labels = {}
threshold = 80 # Set a threshold for similarity

for label in labels:
    found = False
    for key in grouped_labels:
        if fuzz.ratio(label.lower(), key.lower()) > threshold:
            grouped_labels[key].append(label)
            found = True
            break
    if not found:
        grouped_labels[label] = [label]

# grouped_labels["fridge"].append("refrigerator")
# grouped_labels.pop("refrigerator")
grouped_labels, len(grouped_labels)

({'fridge': ['fridge'],
  'dish washer': ['dish washer', 'dishwasher'],
  'microwave': ['microwave'],
  'electric space heater': ['electric space heater',
   'electric heater',
   'electric heater '],
  'electric stove': ['electric stove', 'electric oven'],
  'washer dryer': ['washer dryer'],
  'waste disposal unit': ['waste disposal unit'],
  'ce appliance': ['ce appliance', 'wet appliance'],
  'electric furnace': ['electric furnace'],
  'smoke alarm': ['smoke alarm'],
  'air conditioner': ['air conditioner', 'air conditioning'],
  'subpanel': ['subpanel'],
  'air handling unit': ['air handling unit'],
  'pc': ['pc'],
  'washing machine': ['washing machine', 'washing machine '],
  'dryer': ['dryer'],
  'freezer': ['freezer', 'freezer '],
  'coffee machine': ['coffee machine', 'coffeemachine'],
  'kettle': ['kettle'],
  'laptop': ['laptop', 'laptops'],
  'router': ['router'],
  'air exhaust': ['air exhaust'],
  'stove': ['stove'],
  'stereo': ['stereo'],
  'tablet': ['tablet'],
  'tele

In [62]:
df = pd.read_pickle("./Energy_graph/data/training_data/ECO.pkl")

In [83]:
pd.read_pickle("./Energy_graph/device_mapping.pkl")

{'plug site ': 'plug site ',
 'dryer': 'dryer',
 'food processor': 'food processor',
 'external hard disk': 'external hard disk',
 'coffee machine': 'coffeemachine',
 'coffeemachine': 'coffeemachine',
 'electric stove': 'electric oven',
 'electric oven': 'electric oven',
 'watercooker': 'watercooker',
 'air exhaust': 'air exhaust',
 'monitor': 'monitor',
 'motor': 'monitor',
 'heat living room': 'heat dining room',
 'heat dining room': 'heat dining room',
 'lights bedroom #1 and #3': 'lights bedroom #1 and #3',
 'washing machine': 'washing machine',
 'washing machine ': 'washing machine',
 'pc': 'pc',
 'vacuum cleaner': 'vaccumcleaner',
 'vaccumcleaner': 'vaccumcleaner',
 'gaming pc': 'gaming pc',
 'dish washer': 'dish washer',
 'dishwasher': 'dish washer',
 'fan': 'fan',
 'refrigerator': 'refrigerator',
 'hairdryer straightener': 'hair straighteners',
 'hair straighteners': 'hair straighteners',
 'tumble dryer ': 'tumble dryer',
 'tumble dryer': 'tumble dryer',
 'tumble dryer 3': 'tum